### Differentiation on Linear Subspaces of G

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#2.1
A = create_random_multivector(alg)
B = create_random_multivector(alg)
F = lambda X: (X+B)**3
X = create_random_multivector(alg)
h = 6e-6 # leave h as a global var, so to modify with ease
# central scheme, modify h to minimize truncation/round-off error
def differential(F, X, A):
    d = (h)*A
    return (1/(2*h))*F(X+d) - (1/(2*h))*F(X-d)

In [3]:
# a surgestion of h
def auto_step_size(x):
    """
    Automatically determine step size based on variable magnitude and machine epsilon.
    """
    eps = np.finfo(float).eps
    return np.maximum(np.abs(x) * np.cbrt(eps), 100 * eps)
X = create_random_multivector(alg)
np.mean(auto_step_size(X[:]))

2.738940717739964e-06

In [4]:
#2.3
A2 = create_r_blade(2, alg)
F = lambda X: P(X, A2)**2
max_diff(differential(F, X, A), differential(F, P(X,A2), A))

2.546585164964199e-11

In [5]:
#2.9
h = 6e-6
G = lambda X: (X+A2)**3
GF = lambda X: G(F(X))
max_diff(differential(GF, X, A), differential(G, F(X), differential(F, X, A)))

5.384208634495735e-10

In [6]:
#2.12
h = 6e-6
B = create_random_multivector(alg)
FA = lambda X: differential(F, X, A)
FB = lambda X: differential(F, X, B)
FAB = differential(FA, X, B)
FBA = differential(FB, X, A)
max_diff(FAB, FBA)

7.275957614183426e-12

In [7]:
def derivative(F, X, alg, grade=None, frame=None, r_frame=None):
    if not frame:
        if grade or (grade == 0):
            frame = r_vector_frame(alg.frame, grade)
            r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
        else:
            frame = multi_frame(alg.frame)
            r_frame = reci_frame(alg.frame)
    dF = 0
    for v, r in zip(frame, r_frame):
        dF += r * (differential(F, X, v))
    return dF

In [8]:
#2.13
# Randomly generate several frames to see dF is independent of frame
h = 6e-6
r = 4
dFs = []
for _ in range(5):
    basis = create_r_vectors(r, alg)
    r_basis = reciprocal(basis)
    frame = multi_frame(basis)
    r_frame = multi_frame(r_basis, reverse=True)

    dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
    dFs.append(dF)
[max_diff(dFs[-1], dF) for dF in dFs[:-1]]

[1.0682723765769442e-09,
 1.4231860347730483e-09,
 8.086391995989315e-10,
 7.581468927801738e-10]

In [9]:
# For F defined on A2, we can restrict our bases
# choose a nonorthonormal-to-A2-vector e.g. e1
v1 = P(e1, A2)
# contract the parallel part
v2 = v1 | A2
frame = multi_frame([v1, v2])
r_frame = reci_frame([v1, v2])
dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
dFs[-1], dF

(3.53 + 0.811 𝐞₁ + 1.62 𝐞₂ + 0.873 𝐞₃ + 2.24 𝐞₄ + -0.0202 𝐞₁₂ + 0.0136 𝐞₁₃ + -0.0718 𝐞₁₄ + 0.0489 𝐞₂₃ + -0.0876 𝐞₂₄ + -0.115 𝐞₃₄ + 9.05e-10 𝐞₁₂₃ + -7.28e-10 𝐞₁₂₄ + 8e-10 𝐞₁₃₄ + 6.11e-10 𝐞₂₃₄ + -6.86e-10 𝐞₁₂₃₄,
 3.53 + 0.811 𝐞₁ + 1.62 𝐞₂ + 0.873 𝐞₃ + 2.24 𝐞₄ + -0.0202 𝐞₁₂ + 0.0136 𝐞₁₃ + -0.0718 𝐞₁₄ + 0.0489 𝐞₂₃ + -0.0876 𝐞₂₄ + -0.115 𝐞₃₄ + 2.02e-12 𝐞₁₂₃ + -4.16e-13 𝐞₁₂₄ + -6.86e-12 𝐞₁₃₄ + -8.49e-12 𝐞₂₃₄ + -2.8e-12 𝐞₁₂₃₄)

In [10]:
#2.18
# double differentials lower accuracy
# wise to avoid/replace in computation
# but if we just want to see the equivalence, we can just choose a fairly large h
h = 1e-2
F_ = lambda A: differential(F, X, A)
d_F_ = derivative(F_, A, alg, frame=frame, r_frame=r_frame)
# dF = derivative(F , X, alg, frame=frame, r_frame=r_frame)
max_diff(d_F_, dF), d_F_, dF

(4.5565329287455825e-11,
 3.53 + 0.811 𝐞₁ + 1.62 𝐞₂ + 0.873 𝐞₃ + 2.24 𝐞₄ + -0.0202 𝐞₁₂ + 0.0136 𝐞₁₃ + -0.0718 𝐞₁₄ + 0.0489 𝐞₂₃ + -0.0876 𝐞₂₄ + -0.115 𝐞₃₄ + 4.11e-14 𝐞₁₂₃ + 2.66e-14 𝐞₁₂₄ + -2.31e-13 𝐞₁₃₄ + -2.42e-13 𝐞₂₃₄ + 1.45e-13 𝐞₁₂₃₄,
 3.53 + 0.811 𝐞₁ + 1.62 𝐞₂ + 0.873 𝐞₃ + 2.24 𝐞₄ + -0.0202 𝐞₁₂ + 0.0136 𝐞₁₃ + -0.0718 𝐞₁₄ + 0.0489 𝐞₂₃ + -0.0876 𝐞₂₄ + -0.115 𝐞₃₄ + 2.02e-12 𝐞₁₂₃ + -4.16e-13 𝐞₁₂₄ + -6.86e-12 𝐞₁₃₄ + -8.49e-12 𝐞₂₃₄ + -2.8e-12 𝐞₁₂₃₄)

In [11]:
# 2.19
h = 1e-5
def adjoint1(F, X, A, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = multi_frame(alg.frame)
        r_frame = reci_frame(alg.frame)
    _F = 0    
    for v, r in zip(frame, r_frame):
        _F += r * alg.sp(differential(F, X, v), A)
    return _F

# either works, but this one's slightly better
def adjoint(F, X, A, alg: Algebra, frame=None, r_frame=None):
    _F = derivative(lambda X: alg.sp(F(X), (A)), X, alg, frame=frame, r_frame=r_frame)
    return _F

_F = adjoint(F, X, A, alg)
_F1 = adjoint1(F, X, A, alg)
F_BA = differential(F, X, B).sp(A)

(
max_diff(B.sp(_F), F_BA),
max_diff(B.sp(_F1), F_BA)
)

(4.994316071815774e-11, 4.704236999941713e-11)

In [12]:
#2.20 F defined on A2
max_diff(P(_F, A2), _F)

1.649724801211505e-11

In [13]:
#2.21
[max_diff(_F.grade(r), derivative(lambda X: F(X).sp(A), X, alg, r)) for r in range(0, 5)]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [14]:
#2.24
r = 2
_sum = derivative(lambda X: F(X), X, alg) + derivative(lambda X: G(X), X, alg)
max_diff(derivative(lambda X: F(X) + G(X), X, alg), _sum)

8.748557434046234e-11

In [15]:
#2.25a
_prod = derivative(lambda X: F(X), X, alg)*G(X) + derivative(lambda Y:F(X)*G(Y), X, alg)
max_diff(derivative(lambda X: F(X)*G(X), X, alg), _prod)

1.0557599239291449e-09

In [16]:
#2.25b The note by Hitzer regard this as a definition. But we shall see it's not! The operators have their meaning.
# r-derivative
grade = 2
frame = r_vector_frame(alg.frame, grade)
r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v) # the over dots made clear using lambda function
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

9.840217529699657e-10

In [17]:
#2.25b derivative on entire frame
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.3081677963100447e-09

In [18]:
#2.26 r-derivative
h = 1e-4
r = 2
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

3.717526797508697e-08

In [19]:
#2.26 derivative
h = 1e-4
r = None
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

4.6068072023786044e-08

In [20]:
#2.28a
(
    max_diff(P(A, A2), differential(lambda X: P(X, A2), X, A)),
    max_diff(P(A, A2), derivative(lambda X: P(X, A2).sp(A), X, alg))
)

(1.899369550528718e-12, 1.3642526161549479e-12)

In [21]:
#2.28b
max_diff(P(A, A2).reverse(), differential(lambda X: P(X.reverse(), A2), X, A))

1.899369550528718e-12

In [22]:
#2.29 d of n dim spade is 2^n
h = 1e-5
A3 = create_r_blade(3, alg)
(
derivative(lambda X: X, X, alg), #4d
derivative(lambda X: P(X, A3), X, alg), #3d
derivative(lambda X: P(X, A2), X, alg) #2d
)

(16.0,
 8.0 + 4.55e-12 𝐞₁₂ + 8.64e-12 𝐞₁₃ + -5.91e-12 𝐞₁₄ + -2.73e-11 𝐞₂₃ + 2e-11 𝐞₂₄ + 2.73e-12 𝐞₃₄,
 4.0 + -2.3e-12 𝐞₁₂ + 7.45e-12 𝐞₁₃ + -1.62e-12 𝐞₁₄ + 6.74e-12 𝐞₂₃ + 8.16e-12 𝐞₂₄ + 1.82e-12 𝐞₃₄ + 2.84e-12 𝐞₁₂₃₄)

In [23]:
#2.30
max_diff(derivative(lambda X: X.sp(X.reverse()), X, alg), 2*X.reverse())

1.8210322139111668e-11

In [24]:
#2.31
k=6
max_diff(differential(lambda X: X**k, X, A), sum((X**l)*A*(X**(k-1-l)) for l in range(k)))

1.544280792131758e-08

In [25]:
#2.32 To allow different signature, we consider norm square instead
def normsq(X):
    return X.sp(X.reverse())[0]

k=3    
max_diff(derivative(lambda X: normsq(X) ** (k), X, alg), 2*k*normsq(X)**(k-1)*X.reverse())

1.9907169246380363e-09

In [26]:
#2.33
max_diff(derivative(lambda X: np.log(np.abs(normsq(X))), X, alg), 2 * X.reverse()/normsq(X))

9.0130014562817e-11

In [27]:
#2.30 var
for n in range(7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda X: X * X.reverse(), X, alg)
    Xr = X.reverse()
    ratio = np.round(dXXr[0]/Xr[0])
    print(ratio, max_diff(dXXr, ratio*Xr)) 

2.0 8.704148513061227e-13
4.0 2.5191848607164502e-11
6.0 1.4452439245360438e-11
8.0 2.417666067344726e-11
12.0 1.234825575124887e-10
24.0 1.723599041270063e-10
56.0 5.994280627419357e-10


Compare with 2.2.30. The derivative is also an integer multiple of $\tilde{X}$, yet a complex one:
$$ \partial  X\tilde{X} = \.{\partial} \.{X} \tilde{X} + \.{\partial} X \.{\tilde{X}} = 2^n \tilde{X} + {\gamma}_n \tilde{X}$$
where ${\gamma}_n$ are suggests to be [1, 2, 2, 0, -4, -8, -8] for $0\leq n \leq 6$ from the above and below numerical tests:

In [28]:
for n in range(1,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda Y: X*Y.reverse(), X, alg)
    print(n, dXXr[0]/X[0])

1 1.9999999999907014
2 1.999999999973919
3 1.3128132808867302e-11
4 -3.999999996156127
5 -8.0000000003025
6 -7.999999999606236


$\partial X^2$ is conversely neither a multiple of $X$ nor $\tilde{X}$.
But if we separate scalars or pseudoscalars from other grades, another integer multiple emerges. 
Try proving it!

In [29]:
# 2.30 var2
for n in range(2,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXX = derivative(lambda X: X**2, X, alg)
    ratios = np.round(dXX[0]/X[0])
    ratiov = np.round(dXX[1]/X[1])
    ratiops = np.round(dXX[2**n-1]/X[2**n-1])
    print(ratios, ratiov, ratiops, max_diff(dXX[1:2**n-1], ratiov*(X[1:2**n-1]))) 

8.0 4.0 4.0 1.5476508963274682e-12
16.0 8.0 16.0 2.1952217821308295e-11
32.0 16.0 16.0 6.247979911222501e-11
64.0 32.0 64.0 1.922444425872527e-10
128.0 64.0 64.0 6.024833965057041e-10


In [30]:
#2.37a
from scipy.special import comb
for n, r in [(5,2), (6,2), (1,1), (1,0), (2,0)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    print(derivative(lambda X: X, X, alg, r), comb(n, r, exact=True))

10.0 10
15.0 15
1.0 1
1.0 1
1.0 1


In [31]:
#2.38a
def gamma(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    print(max_diff(derivative(lambda X: Ar*X, X, alg, s), gamma(n, r, s)*Ar)) 

2.6012525466967418e-11
8.497602621559963e-11
9.050760141349201e-12
3.1720182036565348e-12
1.7066348334537906e-12


In [32]:
#2.38b
n = 5
s = 2
alg = Algebra(n)
X = create_random_multivector(alg)
A = create_random_multivector(alg)
max_diff(derivative(lambda X: A*X, X, alg, s), sum(gamma(n, r, s)*A.grade(r) for r in range(0, n+1)))


7.813882874074807e-11

Construct $\gamma_n$ from $\Gamma_s^r$

In [33]:
#2.38a reverse version
def gamma_rev(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k+s*(s-1)/2)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    dAXr = derivative(lambda X: Ar*X.reverse(), X, alg, s)
    print(max_diff(dAXr, gamma_rev(n, r, s)*Ar)) 

1.5052070700960485e-11
9.147216317728635e-11
8.346656699131927e-13
4.667488617826621e-12
4.811706588725428e-13


In [34]:
for n in range(7):
    r_parts = np.zeros(n+1)
    for s in range(0, n+1):
        for r in range(0, n+1):
            r_parts[r] += (-1)**(r*(r-1)/2)*gamma_rev(n, r, s) # one more reversion
    print(r_parts)

[1.]
[2. 2.]
[2. 2. 2.]
[0. 0. 0. 0.]
[-4. -4. -4. -4. -4.]
[-8. -8. -8. -8. -8. -8.]
[-8. -8. -8. -8. -8. -8. -8.]


The coefficients of each grade match due to the reversion-symmetry of gamma_rev.
Thus we get $\gamma_n$ by fixing any r:

In [35]:
def gamma_n(n):
    return sum(gamma_rev(n, 0, s) for s in range(0, n+1))
[gamma_n(n) for n in range(7)]

[1.0, 2.0, 2.0, 0.0, -4.0, -8.0, -8.0]

In [36]:
#3.1a dAG and ddGA are equivalent according to 2.18
h = 1e-3
alg = Algebra(2)
A = create_random_multivector(alg)
B = create_random_multivector(alg)
G = lambda X: (X+B) ** 2

dAG = derivative(lambda A: G(A^B), A, alg)
ddGA = derivative(lambda A_: differential(G, A^B, A_^B), A, alg)

max_diff(dAG, ddGA), dAG, ddGA

(6.644995664828457e-10,
 9.82 + 7.55 𝐞₁ + 6.66 𝐞₂ + 1.45 𝐞₁₂,
 9.82 + 7.55 𝐞₁ + 6.66 𝐞₂ + 1.45 𝐞₁₂)

In [39]:
#3.1b a headache...
h = 1e-2 # sufficiently large to see the equivalence

ddGAB = derivative(lambda B: derivative(lambda A: G(A^B), A, alg), B, alg)
dAdBG = (
    derivative(lambda B_: derivative(lambda A_: differential(G, A^B, A_^B_), A, alg), B, alg) +
    derivative(lambda B_: differential(lambda X: derivative(lambda A_: differential(G, X, A_^B), A, alg), A^B, A^B_), B, alg)
    )
max_diff(ddGAB, dAdBG), ddGAB, dAdBG

(6.810871866491652e-08,
 32.1 + 12.6 𝐞₁ + 11.7 𝐞₂ + 5.46 𝐞₁₂,
 32.1 + 12.6 𝐞₁ + 11.7 𝐞₂ + 5.46 𝐞₁₂)

In [40]:
#3.2b
h = 1e-5
for r in range(1, 7):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = derivative(lambda A: G(A^B), A, alg, grade=s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            BdG += inner(B, vr) * (differential(G, A^B, v))
        assert_diff(dAG, BdG, 1e-8)
        

In [41]:
#3.2c Somehow the pseudoscalars are more ill conditioned, let me know if you figure out
h = 1e-2
for r in range(2,5):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = lambda B: derivative(lambda A: G(A^B), A, alg, grade=s)
        dBdAG = derivative(dAG, B, alg, grade=r-s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            for u, ur in zip(frame, r_frame):
                BdG += inner(A, u) * inner(B, vr) * (differential(lambda X: differential(G, X, ur), A^B, v))
        BdG *= (-1)**(s*(r-s))
        BdG += comb(r, r-s)*derivative(G, A^B, alg, grade=r)

        print(dBdAG, BdG, sep="\n")

0.565 𝐞₁₂
0.451 𝐞₁₂
0.866 + -1.48 𝐞₁₂
0.866 + -1.48 𝐞₁₂
2.16 𝐞₁₂₃
1.86 𝐞₁₂₃
5.33e-13 + -1.27 𝐞₁ + 0.0762 𝐞₂ + 0.841 𝐞₃ + 4.89 𝐞₁₂ + 2.63 𝐞₁₃ + 3.08 𝐞₂₃ + 3.7 𝐞₁₂₃
-3.05e-13 + -1.27 𝐞₁ + 0.0762 𝐞₂ + 0.841 𝐞₃ + 4.89 𝐞₁₂ + 2.63 𝐞₁₃ + 3.08 𝐞₂₃ + 3.7 𝐞₁₂₃
1.93 𝐞₁ + 2.42 𝐞₂ + 3.29 𝐞₃ + -0.958 𝐞₁₂₃
1.93 𝐞₁ + 2.42 𝐞₂ + 3.29 𝐞₃ + -0.958 𝐞₁₂₃
0.568 𝐞₁₂₃₄
0.488 𝐞₁₂₃₄
0.847 𝐞₁₂ + 0.881 𝐞₁₃ + -1.16 𝐞₁₄ + 1.08 𝐞₂₃ + -0.528 𝐞₂₄ + 0.941 𝐞₃₄ + -6.15 𝐞₁₂₃₄
0.847 𝐞₁₂ + 0.881 𝐞₁₃ + -1.16 𝐞₁₄ + 1.08 𝐞₂₃ + -0.528 𝐞₂₄ + 0.941 𝐞₃₄ + -6.15 𝐞₁₂₃₄
-4.64 + 8.25 𝐞₁₂ + 2.35 𝐞₁₃ + 11.6 𝐞₁₄ + 4.98 𝐞₂₃ + 11.0 𝐞₂₄ + 1.58 𝐞₃₄ + -1.83 𝐞₁₂₃₄
-4.64 + 8.25 𝐞₁₂ + 2.35 𝐞₁₃ + 11.6 𝐞₁₄ + 4.98 𝐞₂₃ + 11.0 𝐞₂₄ + 1.58 𝐞₃₄ + -1.83 𝐞₁₂₃₄
1.01 𝐞₁₂ + 0.784 𝐞₁₃ + -0.877 𝐞₁₄ + 1.57 𝐞₂₃ + -0.112 𝐞₂₄ + 1.27 𝐞₃₄ + 4.04 𝐞₁₂₃₄
1.01 𝐞₁₂ + 0.784 𝐞₁₃ + -0.877 𝐞₁₄ + 1.57 𝐞₂₃ + -0.112 𝐞₂₄ + 1.27 𝐞₃₄ + 4.04 𝐞₁₂₃₄


In [42]:
#3.4
h = 1e-3
alg = Algebra(r)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
dAL = lambda B: derivative(lambda A: L(A^B), A, alg, grade=s)
dBdAL = derivative(dAL, B, alg, grade=r-s)
max_diff(dBdAL, comb(r, r-s) * derivative(L, A^B, alg, grade=r))

2.9320190719772654e-10

In [43]:
def multi_differential(F, X, vectors):
    from functools import reduce
    
    def apply_differential(current_func, vector):
        return lambda X: differential(current_func, X, vector)
    
    # Reduce applies the function cumulatively to the items of iterable, from left to right,
    # so as to reduce the iterable to a single value.
    return reduce(apply_differential, vectors, F)(X)

In [44]:
def multi_derivative(F, vectors):
    def apply_derivative(current_func, V):
        assert len(V.grades) == 1, 'vectors not simple'
        return lambda X: derivative(lambda Y: current_func(Y ^ X), V, alg, grade=V.grades[0])
    
    return reduce(apply_derivative, vectors, F)(1)

# Excercise: use frame and differential to split the derivatives. Iterate over multiindexes sounds a good idea

In [45]:
#3.5
from math import factorial
h = 1e-2
alg = Algebra(1,3)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
r = 3
n = len(alg.frame)
frame = alg.frame
r_frame = reciprocal(alg.frame)
vectors = create_r_vectors(r, alg)
X = wedge(vectors)

drL = multi_derivative(L, vectors) 
drL, factorial(r)*derivative(L, X, alg, grade=r)


(33.9 + 35.3 𝐞₁ + 23.0 𝐞₂ + -0.0283 𝐞₃ + -10.5 𝐞₄ + 6.46 𝐞₁₂ + 2.79 𝐞₁₃ + 7.91 𝐞₁₄ + -27.2 𝐞₂₃ + 9.5 𝐞₂₄ + -9.78 𝐞₃₄ + 11.4 𝐞₁₂₃ + -8.84 𝐞₁₂₄ + 14.4 𝐞₁₃₄ + 35.5 𝐞₂₃₄ + 13.8 𝐞₁₂₃₄,
 33.9 + 35.3 𝐞₁ + 23.0 𝐞₂ + -0.0283 𝐞₃ + -10.5 𝐞₄ + 6.46 𝐞₁₂ + 2.79 𝐞₁₃ + 7.91 𝐞₁₄ + -27.2 𝐞₂₃ + 9.5 𝐞₂₄ + -9.78 𝐞₃₄ + 11.4 𝐞₁₂₃ + -8.84 𝐞₁₂₄ + 14.4 𝐞₁₃₄ + 35.5 𝐞₂₃₄ + 13.8 𝐞₁₂₃₄)

In [46]:
# We can also expand the vector derivatives. And a linear function equals to its differential
drL = 0
r = 3
for v, vr in zip(frame, r_frame):
    for u, ur in zip(frame, r_frame):
        for w, wr in zip(frame, r_frame):
            drL += vr * u * w * L(wr^ur^v)
drL

33.9 + 35.3 𝐞₁ + 23.0 𝐞₂ + -0.0283 𝐞₃ + -10.5 𝐞₄ + 6.46 𝐞₁₂ + 2.79 𝐞₁₃ + 7.91 𝐞₁₄ + -27.2 𝐞₂₃ + 9.5 𝐞₂₄ + -9.78 𝐞₃₄ + 11.4 𝐞₁₂₃ + -8.84 𝐞₁₂₄ + 14.4 𝐞₁₃₄ + 35.5 𝐞₂₃₄ + 13.8 𝐞₁₂₃₄

In [47]:
# A cleaner version of above
from itertools import permutations
drL = 0
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    drL += wedge(base_vectors[::-1]) * L(wedge(reci_vectors))
drL

33.9 + 35.3 𝐞₁ + 23.0 𝐞₂ + -0.0283 𝐞₃ + -10.5 𝐞₄ + 6.46 𝐞₁₂ + 2.79 𝐞₁₃ + 7.91 𝐞₁₄ + -27.2 𝐞₂₃ + 9.5 𝐞₂₄ + -9.78 𝐞₃₄ + 11.4 𝐞₁₂₃ + -8.84 𝐞₁₂₄ + 14.4 𝐞₁₃₄ + 35.5 𝐞₂₃₄ + 13.8 𝐞₁₂₃₄

In [51]:
from itertools import product
def vectors_partial(F, vectors, directions):
    r = len(vectors)
    drF = 0
    for offset in product([1,-1], repeat=r):
        offset = np.array(offset)
        coef = np.prod(offset)
        offset = offset * h
        drF += 1/(2*h)**r * coef * F([a+v*d for a, v, d in zip(vectors, directions, offset)])
    return drF

In [52]:
#3.6 simplicial derivative
from itertools import permutations
h = 1e-2
def simplicial_derivative(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += wedge(base_vectors[::-1]) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

simplicial_derivative(lambda vectors: L(wedge(vectors)), vectors, alg) * factorial(r)

33.9 + 35.3 𝐞₁ + 23.0 𝐞₂ + -0.0283 𝐞₃ + -10.5 𝐞₄ + 6.46 𝐞₁₂ + 2.79 𝐞₁₃ + 7.91 𝐞₁₄ + -27.2 𝐞₂₃ + 9.5 𝐞₂₄ + -9.78 𝐞₃₄ + 11.4 𝐞₁₂₃ + -8.84 𝐞₁₂₄ + 14.4 𝐞₁₃₄ + 35.5 𝐞₂₃₄ + 13.8 𝐞₁₂₃₄

In [53]:
#3.7a
h = 1e-2
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(simplicial_derivative(lambda vectors: wedge(vectors), vectors, alg))

1.0
5.0 + 9.7e-12 𝐞₁₂ + -5e-12 𝐞₂₃ + 9.26e-13 𝐞₃₄ + 9.47e-14 𝐞₃₅
10.0 + -3.79e-13 𝐞₁₂ + 5.31e-13 𝐞₁₃ + 3.03e-13 𝐞₂₄ + -1.52e-13 𝐞₂₅ + -6.06e-13 𝐞₃₄ + -1.52e-13 𝐞₃₅
6.0 + 4.61e-12 𝐞₁₂ + 4.37e-12 𝐞₁₃ + 4.85e-12 𝐞₁₄ + 1.39e-11 𝐞₁₅ + 4.62e-13 𝐞₁₆ + 2.24e-12 𝐞₂₅ + -2.06e-13 𝐞₃₅ + 3.4e-12 𝐞₄₅ + 1.36e-13 𝐞₅₆
15.0 + 1.03e-11 𝐞₁₂ + -5.1e-12 𝐞₁₃ + -3.79e-13 𝐞₁₄ + -1.21e-12 𝐞₁₅ + -2.37e-12 𝐞₁₆ + -7.58e-13 𝐞₂₃ + -9.7e-12 𝐞₂₄ + -1.52e-13 𝐞₂₆ + 7.53e-12 𝐞₃₄ + -1.14e-12 𝐞₃₅ + -2.12e-12 𝐞₃₆ + -6.32e-13 𝐞₄₆ + -7.58e-14 𝐞₅₆
20.0 + -8.08e-14 𝐞₁₂ + 7.82e-14 𝐞₁₃ + 3.85e-15 𝐞₁₄ + 5.92e-15 𝐞₁₅ + 3.77e-13 𝐞₁₆ + 9.77e-15 𝐞₂₃ + 9.77e-15 𝐞₂₄ + 4.74e-15 𝐞₂₅ + 3.03e-13 𝐞₂₆ + 1.48e-15 𝐞₃₄ + 3.55e-15 𝐞₃₅ + 3.08e-13 𝐞₃₆ + -1.5e-13 𝐞₄₅ + -2.37e-15 𝐞₄₆ + 5.92e-16 𝐞₅₆


In [54]:
#3.7b X^2 is certainly not a linear function. I gave the r-derivatives of X^2 in 2.30 var2
h = 1e-3
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(max_diff(simplicial_derivative(lambda vectors: wedge(vectors)**2, vectors, alg), (r+1) * wedge(vectors)))

6.781669933952905e-09
6.086138262806351e-08
2.318449821014923e-09
0.00033252927443239155
1.0626735806873455e-06
7.518056920063909e-09


In [55]:
#3.8 Just a ratio of factorials
r, s = 7, 4
factorial(r+s) / (factorial(r) * factorial(s)), comb(r+s, s)

(330.0, 330.0)

In [91]:
#3.9
n = 5
r = 4
h = 1e-5
alg = Algebra(n)
multivectors = [create_random_multivector(alg) for _ in range(r)]
vectors = create_r_vectors(r, alg)
Fr = lambda vectors: sum(multivectors[i] * vectors[i] for i in range(r))

akpkFp = 0
akpkFn = 0
for i in range(r):
    akpkFp += Fr(vectors[:i] +[vectors[i]+vectors[i]*h] + vectors[i+1:])
    akpkFn += Fr(vectors[:i] +[vectors[i]-vectors[i]*h] + vectors[i+1:])

akpkFp *= 1/(2*h)
akpkFn *= 1/(2*h)
akpkF = akpkFp - akpkFn
akpkF, Fr(vectors)


(7.0 + 6.73 𝐞₁ + 4.67 𝐞₂ + 1.91 𝐞₃ + -0.161 𝐞₄ + -4.58 𝐞₅ + 3.35 𝐞₁₂ + 2.36 𝐞₁₃ + 0.0058 𝐞₁₄ + -3.82 𝐞₁₅ + 2.87 𝐞₂₃ + 1.43 𝐞₂₄ + -0.433 𝐞₂₅ + 3.44 𝐞₃₄ + 0.74 𝐞₃₅ + 2.63 𝐞₄₅ + 4.01 𝐞₁₂₃ + 2.2 𝐞₁₂₄ + -0.585 𝐞₁₂₅ + 3.43 𝐞₁₃₄ + -1.47 𝐞₁₃₅ + 1.64 𝐞₁₄₅ + 0.0109 𝐞₂₃₄ + -1.55 𝐞₂₃₅ + -0.00149 𝐞₂₄₅ + -0.732 𝐞₃₄₅ + 0.825 𝐞₁₂₃₄ + -1.94 𝐞₁₂₃₅ + -0.18 𝐞₁₂₄₅ + -0.728 𝐞₁₃₄₅ + 1.68 𝐞₂₃₄₅ + 0.861 𝐞₁₂₃₄₅,
 7.0 + 6.73 𝐞₁ + 4.67 𝐞₂ + 1.91 𝐞₃ + -0.161 𝐞₄ + -4.58 𝐞₅ + 3.35 𝐞₁₂ + 2.36 𝐞₁₃ + 0.0058 𝐞₁₄ + -3.82 𝐞₁₅ + 2.87 𝐞₂₃ + 1.43 𝐞₂₄ + -0.433 𝐞₂₅ + 3.44 𝐞₃₄ + 0.74 𝐞₃₅ + 2.63 𝐞₄₅ + 4.01 𝐞₁₂₃ + 2.2 𝐞₁₂₄ + -0.585 𝐞₁₂₅ + 3.43 𝐞₁₃₄ + -1.47 𝐞₁₃₅ + 1.64 𝐞₁₄₅ + 0.0109 𝐞₂₃₄ + -1.55 𝐞₂₃₅ + -0.00149 𝐞₂₄₅ + -0.732 𝐞₃₄₅ + 0.825 𝐞₁₂₃₄ + -1.94 𝐞₁₂₃₅ + -0.18 𝐞₁₂₄₅ + -0.728 𝐞₁₃₄₅ + 1.68 𝐞₂₃₄₅ + 0.861 𝐞₁₂₃₄₅)

In [92]:
#3.10
h = 1e-2

def skew_symmetrizer(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    Ar = wedge(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += (Ar| wedge(base_vectors[::-1])) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

skew_symmetrizer(Fr, vectors, alg)

-4.63e-10 + -4.26e-10 𝐞₁ + 7e-10 𝐞₂ + 2.86e-10 𝐞₃ + -7.61e-12 𝐞₄ + -3.84e-10 𝐞₅ + -1.7e-10 𝐞₁₂ + 2.16e-10 𝐞₁₃ + -6.7e-13 𝐞₁₄ + -2.09e-10 𝐞₁₅ + 1.27e-10 𝐞₂₃ + -7.63e-11 𝐞₂₄ + 6.86e-11 𝐞₂₅ + 5.91e-11 𝐞₃₄ + 2.17e-11 𝐞₃₅ + 3.9e-10 𝐞₄₅ + 2.15e-10 𝐞₁₂₃ + -6.87e-10 𝐞₁₂₄ + -3.91e-12 𝐞₁₂₅ + 1.37e-10 𝐞₁₃₄ + -2.03e-10 𝐞₁₃₅ + 2.49e-10 𝐞₁₄₅ + 4.78e-12 𝐞₂₃₄ + -1.66e-10 𝐞₂₃₅ + -1.81e-12 𝐞₂₄₅ + -5.11e-11 𝐞₃₄₅ + 1.72e-11 𝐞₁₂₃₄ + 8.73e-12 𝐞₁₂₃₅ + 4.91e-12 𝐞₁₂₄₅ + 6.48e-11 𝐞₁₃₄₅ + 1.83e-11 𝐞₂₃₄₅ + 3.87e-11 𝐞₁₂₃₄₅

In [81]:
#3.11
h = 1e-2
# skew_symmetrizer(lambda vectors: skew_symmetrizer(Fr, vectors, alg), vectors, alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda vectors: C * wedge(vectors) * D
skew_symmetrizer(L, vectors, alg), L(vectors)

(0.153 + 0.00643 𝐞₁ + 0.0343 𝐞₂ + 0.0211 𝐞₃ + 0.206 𝐞₁₂ + -0.119 𝐞₁₃ + 0.0558 𝐞₂₃ + 0.123 𝐞₁₂₃,
 0.153 + 0.00643 𝐞₁ + 0.0343 𝐞₂ + 0.0211 𝐞₃ + 0.206 𝐞₁₂ + -0.119 𝐞₁₃ + 0.0558 𝐞₂₃ + 0.123 𝐞₁₂₃)

In [120]:
def blade_split(Ar, alg):
    r = Ar.grades[0]
    projects = [P(e, Ar) for e in alg.frame[:r]]
    wed = wedge(projects)
    ratio = Ar[:][0]/wed[:][0]
    return projects, ratio

In [123]:
#3.12
r = 3
Ar = create_r_blade(r, alg)
alpha = lambda vectors: wedge(vectors) | Ar
vectors, ratio = blade_split(Ar, alg)
vectors[0] *= ratio
max_diff(simplicial_derivative(alpha, vectors, alg), Ar)

2.1007751094259675e-11